Creates corner plots of Air Quality variables with mortality and poverty variables. 


In [7]:
###Edit this to your system
homedir = "/Users/ennesser.1//Documents/Erdos22/Capstone-/"

In [8]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import corner

In [9]:
def flatten_df(df, filter_by = {}, save_fn = None): 
    
    sort_by = {'Gender Code' : None,
               'Race' : None,
               'Hispanic Origin' : None,
               'Interval' : None,
               'Age Group Code': None,
               'State' : None}
    county_level_cols = ['County','5_Year_Avg_Poverty_Estimate','Total 5yrAvg County Population',
                         'Total 5yrAvg County Population','Total 5yrAvg County Deaths',
                         'Total 5yrAvg State Population','Total 5yrAvg State Deaths',
                         'Total 15yrAvg County Population','Total 15yrAvg County Deaths',
                         'Total 15yrAvg State Population','Total 15yrAvg State Deaths','15_Year_Avg_Poverty_Estimate',
                         'CO 2nd Max 1-hr', 'CO 2nd Max 8-hr','NO2 98th Percentile 1-hr', 
                         'NO2 Mean 1-hr', 'Ozone 2nd Max 1-hr', 'Ozone 4th Max 8-hr', 
                         'SO2 99th Percentile 1-hr', 'SO2 2nd Max 24-hr','SO2 Mean 1-hr', 
                         'PM2.5 98th Percentile 24-hr', 'PM2.5 Weighted Mean 24-hr', 
                         'PM10 2nd Max 24-hr', 'Lead Max 3-Mo Avg']
    air_q_cols = ['CO 2nd Max 1-hr', 'CO 2nd Max 8-hr','NO2 98th Percentile 1-hr', 
                  'NO2 Mean 1-hr', 'Ozone 2nd Max 1-hr', 'Ozone 4th Max 8-hr', 
                  'SO2 99th Percentile 1-hr', 'SO2 2nd Max 24-hr','SO2 Mean 1-hr', 
                  'PM2.5 98th Percentile 24-hr', 'PM2.5 Weighted Mean 24-hr', 
                  'PM10 2nd Max 24-hr', 'Lead Max 3-Mo Avg']
    
    # Check for bad input
    for key in filter_by.keys(): 
        if key not in sort_by: 
            print('Unrecognized key in filter_by dict.')
            return
    
    for key in filter_by:
        if filter_by[key] != None: sort_by[key] = filter_by[key]
    
    # Deal with interval issues
    temp_df = df.copy(deep=True)
    if sort_by['Interval'] != None: 
        num_years = 5
        mask = np.asarray(temp_df['Interval']==sort_by['Interval'])
        temp_df = temp_df.loc[mask]
        
    else: 
        num_years = 15
        fips = np.unique(temp_df['FIPS'].astype(str))
        for fip in fips:
            temp = temp_df.loc[temp_df['FIPS']==fip]
            all_ints,ind = np.unique(temp['Interval'],return_index=True)
            for quant in air_q_cols:
                mask = temp_df['FIPS'] == fip
                temp_df.loc[mask,quant] = np.nanmean(temp.iloc[ind][quant])

    # Remove all unnecesary data from the df
    for key,val in sort_by.items():
        if val == None: 
            temp_df = temp_df.drop(columns=key)
        else:    
            temp_df = temp_df.loc[np.asarray(temp_df[key]==val)]
            
    # Flatten remaining data points so that there are no repeated counties/FIPS
    fips,ind = np.unique(temp_df['FIPS'].astype(str),return_index=True)
    
    ### Get values that are constant for different groups in the same county
    county_levels = {}
    for col in county_level_cols:
        county_levels[col] = np.asarray(temp_df[col])[ind]
        
    ### Calculate the values that change with different groups in the same county
    deaths = []
    population = []
    for fip in fips:
        mask = np.asarray(temp_df['FIPS']==fip)
        single_county_df = temp_df.loc[mask]
        deaths.append(np.sum(single_county_df['Deaths']))
        population.append(np.sum(single_county_df['Population']))
    
    ### Calculate deaths and population per year
    deaths = np.asarray(deaths)/num_years
    population = np.asarray(population)/num_years
    
    # Construct dataframe
    flattened_df = pd.DataFrame(data={'FIPS': fips, 'Deaths': deaths, 'Population': population})
    for col in county_level_cols:
        if sort_by['Interval'] != None:
            flattened_df[col] = county_levels[col]
        elif ' 5yrAvg' not in col and '5_Year_Avg_Poverty_Estimate' != col:
            flattened_df[col] = county_levels[col]
    
    # Save to CSV if necessary
    if save_fn != None:
        flattened_df.to_csv(save_fn)
    
    return flattened_df # dataframe with single row for each county containing deaths, population, poverty, and polution

In [10]:
###Flattens mortality data
#If you have an error, check that mort_airq_poverty.csv is unzipped.
mort = pd.read_csv(homedir + 'data/mort_airq_poverty.csv',dtype={"FIPS": str})
mort_flattened = flatten_df(mort,{'Race':'White','Interval':'2012-2016'})

KeyboardInterrupt: 

In [5]:
###Add rate columns to data
mort_flattened.loc[:,'PovertyRate'] = mort_flattened.loc[:,'5_Year_Avg_Poverty_Estimate']/mort_flattened.loc[:,'Total 5yrAvg County Population']
mort_flattened.loc[:,'DeathRate'] = (mort_flattened.loc[:,'Total 5yrAvg County Deaths']/mort_flattened.loc[:,'Total 5yrAvg County Population'])
mort_flattened2 = mort_flattened[mort_flattened.PovertyRate<=1].reset_index(drop=True)
mort_flattened_log = mort_flattened2[~((mort_flattened2.DeathRate==0) | (mort_flattened2.PovertyRate==0))]

In [1]:
###Makes the corner plots. Only one air quality variable should be selected at a time. Selecting more than one
###will mask too much data.

#Approximate annual averages of the six air quality 
annuals = ['CO 2nd Max 8-hr', 'NO2 Mean 1-hr', 'Ozone 4th Max 8-hr', 'SO2 Mean 1-hr', 'PM2.5 Weighted Mean 24-hr']

#Select the air quality variable you want to look at
aqVar = annuals[2]

povrate = mort_flattened_log['PovertyRate'].to_numpy()
deathrate = mort_flattened_log['DeathRate'].to_numpy()
totalpop = mort_flattened_log['Total 5yrAvg County Population'].to_numpy()
aqMean = mort_flattened_log[aqVar].to_numpy()
#mask out counties that do not have a measurement for this variable.
mask_aqNan = np.logical_not(np.isnan(aqMean))

povrate = povrate[mask_aqNan]
deathrate = deathrate[mask_aqNan]
aqMean = aqMean[mask_aqNan]
totalpop = totalpop[mask_aqNan]

cordata = np.vstack([aqMean,povrate,np.log10(totalpop),deathrate])

corner.corner(data=cordata.T, labels=[aqVar, 'Log PovertyRate','Total Log Population','Log DeathRate'])

NameError: name 'mort_flattened_log' is not defined